In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.functions import udf, col

In [3]:
sc = SparkContext.getOrCreate()
sql=SparkSession(sc)
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [5]:
data=sql.read.csv('HistoricalQuotes.csv',header=True)
data.show(5)

+----------+-----------+-------+----------+----------+----------+
|      Date| Close/Last| Volume|      Open|      High|       Low|
+----------+-----------+-------+----------+----------+----------+
|  06-01-20| $1,431.82 |1217140|$1,418.39 |$1,437.96 |   $1,418 |
|05/29/2020| $1,428.92 |1838059|$1,416.94 |$1,432.57 |$1,413.35 |
|05/28/2020| $1,416.73 |1693976|$1,396.86 |$1,440.84 |   $1,396 |
|05/27/2020| $1,417.84 |1686142|$1,417.25 |$1,421.74 |$1,391.29 |
|05/26/2020| $1,417.02 |2060643|$1,437.27 |   $1,441 |$1,412.13 |
+----------+-----------+-------+----------+----------+----------+
only showing top 5 rows



In [6]:
def clear_string(value : str) -> str:
    value = value.replace('$', '')
    return float(value)
clear_string_udf = udf(lambda value: clear_string(value), DoubleType())

In [7]:
data.printSchema()

root
 |-- Date: string (nullable = true)
 |--  Close/Last: string (nullable = true)
 |--  Volume: string (nullable = true)
 |--  Open: string (nullable = true)
 |--  High: string (nullable = true)
 |--  Low: string (nullable = true)



In [10]:
data= data.withColumn("Date", data["Date"].cast("string")
                                        ).withColumn(" Close/Last", clear_string_udf(col(" Close/Last"))
                                        ).withColumn(" Volume", clear_string_udf(col(" Volume"))
                                        ).withColumn(" Open", clear_string_udf(col(" Open"))
                                        ).withColumn(" High", clear_string_udf(col(" High"))
                                        ).withColumn(" Low", clear_string_udf(col(" Low")))

In [11]:
data.printSchema()

root
 |-- Date: string (nullable = true)
 |--  Close/Last: double (nullable = true)
 |--  Volume: double (nullable = true)
 |--  Open: double (nullable = true)
 |--  High: double (nullable = true)
 |--  Low: double (nullable = true)

